In [1]:
import pyowm
 
class Openwinterface():
    def __init__(self, idioma, cidade,pais,chave):
 
        #retorna um dicionário com os dados do openweather
         
        self.retorno = { }
        self.idioma = idioma
        self.cidade = cidade
        self.pais = pais
        self.chave = chave
 
        # só continua se a chave for uma chave válida
        if self.chave[0] == 'x':
            return
 
        self.get_weather()
 
 
    def get_weather(self):
 
        owm = pyowm.OWM(self.chave,language=self.idioma )   # aqui criamos a nossa interface com a biblioteca
         
        posicao = "%s,%s" % (self.cidade,self.pais)        # definimos o local aonde serão obtidos os dados
         
        observation = owm.weather_at_place(posicao)
         
        dados_clima = observation.get_weather()           # lemos todos os dados disponíveis
         
        # aqui convertemos os dados em um dicionario para retornamos todos os dados disponíveis.           
 
        self.retorno['temperatura']  = dados_clima.get_temperature('celsius')
        self.retorno['vento'] = dados_clima.get_wind()
        self.retorno['umidade'] = dados_clima.get_humidity()
        self.retorno['chuva'] = dados_clima.get_rain()
        self.retorno['pressao'] = dados_clima.get_pressure()
        self.retorno['nascer_sol'] = dados_clima.get_sunrise_time('iso')
        self.retorno['por_sol'] = dados_clima.get_sunset_time('iso')
        return self.retorno

In [ ]:
import telebot
import pycep_correios
import json
from datetime import datetime

bot = telebot.TeleBot("887122726:AAEJ8RzaIo-bRG8f05QNuf9AMmmMFA5bFeQ")

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "/cep Para pesqisar pelo CEP \nInforme o nome da cidade, sigla do pais para consulta (ex.: Goiânia, BR)")
    
@bot.message_handler(commands=['cep'])
def send_welcome(message):
    try:
        cep = pycep_correios.consultar_cep(message.text)
        print(cep)
        t = Openwinterface('pt_br', cep['cidade'], 'BR', '6d3f5dd84b864d9de4e120f9343bf7fc')
        d = t.get_weather()
        print(d['temperatura']['temp'])
        ms = 'Temperatura atual em ' + cep['cidade'] + ': ' + str(d['temperatura']['temp']) + 'º'
        bot.reply_to(message, ms)

        dados = {}
        with open('data/dados.json') as arq:
            dados.update(json.load(arq))

        data = {}
        data['cidade'] = cep['cidade']
        data['temperatura'] = str(d['temperatura']['temp']) + 'º'
        data['datahora'] = str(datetime.now())

        dados['dados'].append(data)

        with open('data/dados.json', 'w') as arq:
            json.dump(dados, arq)
    except:
        print('CEP não encontrado!')
        bot.reply_to(message, 'CEP não encontrado!')
    
@bot.message_handler(func=lambda message: True)
def echo_all(message):
    try:
        x = message.text.split(', ')
        print(x[0], x[1])
        t = Openwinterface('pt_br', x[0], x[1], '6d3f5dd84b864d9de4e120f9343bf7fc')
        d = t.get_weather()
        print(d['temperatura']['temp'])
        ms = 'Temperatura atual em ' + x[0] + ': ' + str(d['temperatura']['temp']) + 'º'
        bot.reply_to(message, ms)
        
        dados = {}
        with open('data/dados.json') as arq:
            dados.update(json.load(arq))

        data = {}
        data['cidade'] = cep['cidade']
        data['temperatura'] = str(d['temperatura']['temp']) + 'º'
        data['datahora'] = str(datetime.now())

        dados['dados'].append(data)

        with open('data/dados.json', 'w') as arq:
            json.dump(dados, arq)
    except:
        bot.reply_to(message, 'Cidade não encontrada!')

bot.polling()